<a href="https://colab.research.google.com/github/ektaarora3501/Neural-Networks/blob/master/Batch_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu

Running the same network but now by using batch normalization.

In [0]:

import tensorflow as tf
tf.reset_default_graph()
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
training=tf.placeholder_with_default(False, shape=(),name="training")


def neuron_layer(X, n_neurons, name, activation=None):
      with tf.name_scope(name):
          n_inputs = int(X.get_shape()[1])
          stddev = 2 / np.sqrt(n_inputs)
          init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
          W = tf.Variable(init, name="kernel")
          b = tf.Variable(tf.zeros([n_neurons]), name="bias")
          Z = tf.matmul(X, W) + b
          if activation is not None:
                 return activation(Z) 
          else:
                 return Z

In [3]:
with tf.name_scope("dnn"):
  hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
  bn1=tf.layers.batch_normalization(hidden1,training=training,momentum=0.9)
  bn1_activation=tf.nn.elu(bn1)
  hidden2 = tf.layers.dense(bn1_activation, n_hidden2, name="hidden2")
  bn2=tf.layers.batch_normalization(hidden2,training=training,momentum=0.9)
  bn2_activation=tf.nn.elu(bn2)
  logits_before = tf.layers.dense(bn2, n_outputs, name="outputs")
  logits=tf.layers.batch_normalization(logits_before,training=training,momentum=0.9)

W0905 02:34:41.918064 139766755895168 deprecation.py:323] From <ipython-input-3-d5d550681bd1>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0905 02:34:41.930074 139766755895168 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0905 02:34:42.213900 139766755895168 deprecation.py:323] From <ipython-input-3-d5d550681bd1>:3: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDAT

In [0]:
with tf.name_scope("loss"):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
  loss = tf.reduce_mean(xentropy, name="loss")

In [5]:
learning_rate = 0.01
with tf.name_scope("train"):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
  correct = tf.nn.in_top_k(logits, y, 1)
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
  print(correct)

init = tf.global_variables_initializer()
saver = tf.train.Saver()  

Tensor("eval/in_top_k/InTopKV2:0", shape=(?,), dtype=bool)


In [6]:
extra_update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
n_epochs = 40
batch_size = 50
with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples//batch_size):
        X_batch,y_batch=mnist.train.next_batch(batch_size)
        sess.run([training_op,extra_update_ops],feed_dict={training:True,X:X_batch,y:y_batch})
    #acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
    acc_test=accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
    
    print(epoch,"test accuracy :",acc_test)
  save_path=saver.save(sess, "./my_model_final.ckpt")
    

W0905 02:34:55.480119 139766755895168 deprecation.py:323] From <ipython-input-6-c7e321cf5a56>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0905 02:34:55.481612 139766755895168 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0905 02:34:55.482839 139766755895168 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0905 02:34:56.567040 139766755895168 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded 

W0905 02:34:56.959431 139766755895168 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 test accuracy : 0.9151
1 test accuracy : 0.9328
2 test accuracy : 0.9484
3 test accuracy : 0.9538
4 test accuracy : 0.9603
5 test accuracy : 0.9617
6 test accuracy : 0.9638
7 test accuracy : 0.9677
8 test accuracy : 0.9701
9 test accuracy : 0.9708
10 test accuracy : 0.9731
11 test accuracy : 0.9706
12 test accuracy : 0.9731
13 test accuracy : 0.9748
14 test accuracy : 0.9731
15 test accuracy : 0.9729
16 test accuracy : 0.9764
17 test accuracy : 0.9736
18 test accuracy : 0.9777
19 test accuracy : 0.9763
20 test accuracy : 0.9759
21 test accuracy : 0.9783
22 test accuracy : 0.9756
23 test accuracy : 0.9791
24 test accuracy : 0.9779
25 test accuracy : 0.9775
26 test accuracy : 0.9763
27 test accuracy : 0.9789
28 test accuracy : 0.9788
29 test accuracy : 0.9799
30 test accuracy : 0.9787
31 test